In [1622]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.quantum_info import Statevector, Operator, partial_trace
from qiskit.circuit.library.standard_gates import RYGate, RZGate, CPhaseGate
import matplotlib.pyplot as plt
import numpy as np
import pylatexenc

## Quantum State Preparation

Takes a $2^n$ dimensional vector $\ket{\psi}\in\mathbb{C}^{2^n}$ such that $||\psi||_2=1$, and outputs a circuit $U$ such that:

$$U\ket{0}_n = \sum_{x=0}^{2^n-1}\psi_x\ket{x}_n$$

It does this by iteratively building states:

$$\ket{\psi} = \sum_{x\in\mathbb{F_2^n}}\psi_{x0}\ket{x}_{n-1}\ket{0} + \psi_{x1}\ket{x}_{n-1}\ket{1}$$

So in order to prepare state $\ket{\psi}_n$, we must first prepare state $\ket{\psi}_{n-1}$, $\ket{\psi}_{n-2}$, $\dots$, $\ket{\psi}_{1}$. 

A quantum state can, in general, be represented as:
\begin{align}
\ket{\psi} = \cos{\frac{\theta}{2}}\ket{0} +e^{i\phi}\sin{\frac{\theta}{2}\ket{1}}
\end{align}

So to turn an intial state of $\ket{0}$ into the desired state of $\ket{\psi} = \alpha\ket{0} +\beta\ket{1}$, we simply have to apply a Y rotation gate with angle $\theta$, and a phase gate with angle $\phi$ to $\ket{0}$. 

However, if both $\ket{0}$ and $\ket{1}$ can have complex coefficients, we need to consider the global phase of the state

Let $\ket{\psi}= \alpha\ket{0} +\beta\ket{1}$ be a state with complex coefficients such that:

$$\ket{\psi} = (a+bi)\ket{0} +(c+di)\ket{1}$$

To remove the imaginary part from the coefficient of $\ket{0}$, we first multiply the state by its global phase $e^{i\gamma} = \frac{a-bi}{\sqrt{a^2+b^2}}$ (so for $\gamma = \frac{1}{i}\ln{\frac{a-bi}{\sqrt{a^2+b^2}}}$):

$$e^{i\gamma}\ket{\psi} = \sqrt{a^2+b^2}\ket{0} +\frac{(a-bi)(c+di)}{\sqrt{a^2+b^2}}\ket{1}$$

Then we will find the angles $\theta$ and $\phi$ to put $\ket{\psi}$ into the form in Equation (1).


We see that $\cos{\frac{\theta}{2}} = \sqrt{a^2+b^2}$, and thus $\theta = 2\arccos{\sqrt{a^2+b^2}}$.

Then to find $\phi$:
\begin{align*}
e^{i\phi}\sin{\frac{\theta}{2}}&=\frac{(a-bi)(c+di)}{\sqrt{a^2+b^2}}\\
e^{i\phi}\sqrt{1-(a^2+b^2)}&=\frac{(a-bi)(c+di)}{\sqrt{a^2+b^2}}\\
e^{i\phi}\sqrt{c^2+d^2}&=\frac{(a-bi)(c+di)}{\sqrt{a^2+b^2}}\\
e^{i\phi}&=\frac{(a-bi)(c+di)}{\sqrt{a^2+b^2}\sqrt{c^2+d^2}}\\
\phi &= \frac{1}{i}\ln{\frac{(a-bi)(c+di)}{\sqrt{a^2+b^2}\sqrt{c^2+d^2}}}
\end{align*}

So:
$$e^{i\gamma}\ket{\psi} = \cos{\frac{\theta}{2}}\ket{0} +e^{i\phi}\sin{\frac{\theta}{2}}\ket{1}$$
where:
- $\gamma = \frac{1}{i}\ln{\frac{\overline{a}}{|a|}}$
- $\theta = 2\arccos{\frac{\overline{a}a}{|a|\sqrt{|a|^2+|b|^2}}}$
- $\phi = \frac{1}{i}\ln{\frac{\overline{a}b}{|a|\sin{\frac{\theta}{2}}}}$

### Toy Example

Here we demonstrate how the code works for the toy example $n=3$:

$$\ket{\psi}_3 = a\ket{000} + b\ket{100} + c\ket{010} + d\ket{110} + e\ket{001} + f\ket{101} + g\ket{011} + h\ket{111}$$

This state is entered into the main function *make_circuit* as a vector $[a, b, c, d, e, f, g, h]$.

- Get angles
    - For the sake of brevity, we demonstrate the angle computation as if the coefficients of $\ket{\psi}$ were real numbers. If the coefficients were complex numbers, we simply add in the computations of angles $\phi$ and $\gamma$ as defined above.
    - Call function *get_angles* on $\ket{\psi}_3$. This references every other entry of the vector (in this case, $a, c, e, g$) in order to build the angles of rotation:
        - $\theta_{200} = 2\arccos{\frac{a}{\sqrt{a^2+b^2}}}$
        - $\theta_{210} = 2\arccos{\frac{c}{\sqrt{c^2+d^2}}}$
        - $\theta_{201} = 2\arccos{\frac{e}{\sqrt{e^2+f^2}}}$
        - $\theta_{211} = 2\arccos{\frac{g}{\sqrt{g^2+h^2}}}$
    
        It also builds the state $\ket{\psi}_2 = \sqrt{a^2+b^2}\ket{00} + \sqrt{c^2+d^2}\ket{10} + \sqrt{e^2+f^2}\ket{01} + \sqrt{g^2+h^2}\ket{11}$, the previous step in the iterative process.

        It returns both the list of angles as well as this new state.

    - Call function *get_angles* on $\ket{\psi}_2$ to get angles of rotation:
        - $\theta_{10} = 2\arccos{\frac{\sqrt{a^2+b^2}}{\sqrt{a^2+b^2+c^2+d^2}}}$
        - $\theta_{11} = 2\arccos{\frac{\sqrt{e^2+f^2}}{\sqrt{e^2+f^2+g^2+h^2}}}$

        And $\ket{\psi}_1 = \sqrt{a^2+b^2+c^2+d^2}\ket{0} + \sqrt{e^2+f^2+g^2+h^2}\ket{1}$

    - Call function *get_angles* on $\ket{\psi}_1$ to get angle of rotation:
        - $\theta_0 = 2\arccos{\sqrt{a^2+b^2+c^2+d^2}}$

- Make list of basis states
    - Call the *make_list* function to make a list of all the $1,...,n-2,n-1$ basis states. For this toy example, this gives: $[[0], [0, 1], [00, 01, 10, 11]]$.

    This list and its sublists correspond to the qubits associated with the $\theta$ angles of rotation. For instance, the rotation of the $\theta_{210}$ angle is associated with the $01$ value in the list. Since the first qubit of this is a zero, we apply an X gate to that qubit, apply a multi-controlled Y gate controlled on both these qubits and applied to the last qubit using $\theta_{210}$ as its angle of rotation, then a multi-controlled Z gate with angle $\gamma_{210}$, then a multi-controlled phase gate with angle $\phi_{210}$, and lastly another multi-controlled phase gate with angle $-\gamma_{210}$, and then uncompute the previously applied X gate.

    If the coefficients of $\ket{\psi}$ are real numbers, the $\phi$ and $\gamma$ angles are all zero, and so applying their respective gates does nothing.

- Make the circuit
    - Initialize the circuit $U$ on $n$ qubits.
    - Apply a Y rotation on the last qubit.

    - Run through the list of angles and corresponding basis states, apply an X gate if a qubit is zero, apply the gates to the qubits, and then uncompute the X gate.

### Code And Demonstration

In [ ]:
def make_list(L,n1):
    bitslist = []
    for j1 in range(L):
        bitslist.append(format(j1,'0'+str(n1)+'b'))
    return bitslist

def get_all_angles(state):
    """
    Iteratively build a list of angles of rotation.
    #Get the angles of rotation to be used lastly in the iterative building process, as well as the coefficients of the state |ψ>_{n-1} (the state we must prepare in order to iteratively build |ψ>_n)
    
    Args:
    - state: A vector of coefficients of the state.
    
    Returns:
    - A list of all the angles of rotation for the circuit.
    """
    n = int(np.log2(len(state)))
    def get_angles(state_0): #Finds the rotation angles of a given state. Outputs a list of angles as well as a new state
        thetas = []
        phis = []
        gammas = []
        newstate = []
        for j1 in range(len(state_0)//2):
            globalphase = (np.conj(state_0[2*j1])) / (np.linalg.norm(state_0[2*j1]))
            theta = 2*np.arccos(globalphase*state_0[2*j1]/np.sqrt(np.linalg.norm(state_0[2*j1])**2+np.linalg.norm(state_0[2*j1+1])**2))


            gamma = np.log(globalphase)/1j
            phi = np.log((globalphase*state_0[2*j1+1])/np.sin(theta/2))/1j

            if np.imag(theta) <= 1e-5:
                thetas.append(np.real(theta))

            phis.append(np.real(phi))

            if np.imag(gamma) <= 1e-5:
                gammas.append(np.real(2*gamma))
            
            newstate.append(np.sqrt(np.linalg.norm(state_0[2*j1])**2+np.linalg.norm(state_0[2*j1+1])**2))
        return thetas, phis, gammas, newstate

    tot_thetas = []; tot_phis = []; tot_gammas = []; #Start with empty lists
    [thetas, phis, gammas, newstate] = get_angles(state)
    tot_thetas.append(thetas); tot_phis.append(phis); tot_gammas.append(gammas); #Add these angles to the list
    for j1 in range(n-1):
        #Get the angles of rotation to be used on the new state, as well as the coefficients of the next state down in the iterative process
        [thetas, phis, gammas, newstate] = get_angles(newstate)
        tot_thetas.append(thetas); tot_phis.append(phis); tot_gammas.append(gammas)
    
    #We will perform this iterative process starting from |ψ>_1 and going up to state |ψ>_n, so we must reverse the order of the angles in the list.
    tot_thetas = tot_thetas[::-1]; tot_phis = tot_phis[::-1]; tot_gammas = tot_gammas[::-1]
    return tot_thetas, tot_phis, tot_gammas


In [ ]:
def make_circuit(psi):
    """
    Takes a vector |ψ> and builds a circuit that will prepare that state.
    
    Args:
    - psi: A vector input of a desired state |ψ>
    
    Returns:
    - U_circ: The quantum circuit that prepares the state |ψ>
    """

    L = len(psi)
    n = int(np.log2(L))

    #Get the angles and phases to be used in the circuit
    [thetas,phis,gammas] = get_all_angles(psi)
    #Make a list of the basis states for n-1,...,1 (example for n = 3: [[00, 01, 10, 11] , [0, 1], [0]]). This is how we will determine where to put X gates in the circuit
    basis_states = []
    for j1 in range(1,n+1):
        basis_states.append(make_list(2**(n-j1),n-j1))

    #Again, since we are performing this iterative process starting with |ψ>_1 and going up to state |ψ>_n, we reverse the order of this list.
    basis_states=list(reversed(basis_states))

    #Initialize the circuit.
    q_reg = QuantumRegister(n, name = "x") #We need as many bits as there are in |ψ>_n
    U_circ = QuantumCircuit(q_reg, name = "Quantum State Preparation")

    #Perform the first Y rotation on the first qubit
    U_circ.ry(thetas[0][0],q_reg[n-1])    

    #For the remainder of the rotations, we will go through the list of angles and apply them to the circuit with a controlled Y rotation gate.
    for j1 in range(1,len(thetas)):
        curr_thetas = thetas[j1]
        curr_phis = phis[j1]
        curr_gammas = gammas[j1]
        for j2 in range(len(curr_thetas)):
            now_theta = curr_thetas[j2]
            now_phi = curr_phis[j2]
            now_gamma = curr_gammas[j2]

            #If the state corresponding to this angle has a zero qubit in it, we apply an X gate to that qubit both before and after the rotation, to ensure that the Y rotation applies correctly.
            for j3 in range(len(basis_states[j1][j2])):
                if basis_states[j1][j2][j3] == '0':
                    U_circ.x(q_reg[n-1-j3])

            
            #Multi-controlled Y gate, Z, gate, and two phase gates
            U_circ.append(RYGate(now_theta).control(j1),q_reg[::-1][0:j1+1])
            U_circ.append(RZGate(now_gamma).control(j1),q_reg[::-1][0:j1+1])
            U_circ.append(CPhaseGate(now_phi).control(j1-1),q_reg[::-1][0:j1+1])
            U_circ.append(CPhaseGate(-now_gamma).control(j1-1),q_reg[::-1][0:j1+1])

            #Uncompute previously applied X gates
            for j3 in range(len(basis_states[j1][j2])):
                if basis_states[j1][j2][j3] == '0':
                    U_circ.x(q_reg[n-1-j3])
            U_circ.barrier()

    return U_circ

In [1677]:
n=3
psi = (1 + np.arange(2**n)) + (1 + np.arange(2**n))*1j
psi=psi/np.linalg.norm(psi)
print('|ψ>',psi)
qc=make_circuit(psi)
st=Statevector(qc)
print('Error',np.linalg.norm(st-psi))

|ψ> [0.04950738+0.04950738j 0.09901475+0.09901475j 0.14852213+0.14852213j
 0.19802951+0.19802951j 0.24753689+0.24753689j 0.29704426+0.29704426j
 0.34655164+0.34655164j 0.39605902+0.39605902j]
Error 1.2276156489239667e-15
